# Import

In [32]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
load_dotenv()
import os
OPENAI = os.getenv("OPENAI_KEY")
from openai import OpenAI
from tqdm import tqdm
import json
import re


### Cleaning

In [70]:
# Clean the unfiltered data
def clean_df(df: pd.DataFrame) -> pd.DataFrame:
    try:
        for elem in df["reviews"]:
            for review in elem:
                try:
                    del review["link"]
                    del review["source"]
                    del review["review_id"]
                    del review["user"]
                    del review["images"]
                except:
                    print("Key not found")
                    pass
    except Exception as e:
        print(e)
        df["reviews"] = "none"
        pass

    df.drop(["business_status", "icon", "icon_background_color", "icon_mask_base_uri", "opening_hours"], inplace=True, axis=1)    
    

    return df

In [71]:
df = pd.read_json("data-with-reviews.json")
# (df["reviews"] == "none").value_counts()
    # 50 rows have no reviews
df = df[df["reviews"] == "none"]
df = clean_df(df)

Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key no

In [72]:
# # extract zip code
df['postal_code'] = df['formatted_address'].apply(lambda x: re.findall(r'\b\d{5}\b', x)[0] if re.findall(r'\b\d{5}\b', x) else None)
df.drop("formatted_address", inplace=True, axis=1)

# drop photos (all unique and non null)
df.drop(["photos"], inplace=True, axis=1)

# topics to boolean
df.loc[df["topics"] == "no topics", "topics"] = False
df.loc[df["topics"] != False, "topics"] = True

# extract lat and long
df['lat'] = df['geometry'].apply(lambda x: x['location']['lat'] if 'location' in x and 'lat' in x['location'] else None)
df['lng'] = df['geometry'].apply(lambda x: x['location']['lng'] if 'location' in x and 'lng' in x['location'] else None)
df.drop("geometry", inplace=True, axis=1)

# greater than 50 total reviews
df['greater_than_50'] = df['user_ratings_total'] > 50

# has certain keywords in "types"
df['is_dentist'] = df['types'].apply(lambda x: 'dentist' in x)
df['is_health'] = df['types'].apply(lambda x: 'health' in x)
df['is_establishment'] = df['types'].apply(lambda x: 'establishment' in x)
df['is_doctor'] = df['types'].apply(lambda x: 'doctor' in x)
df.drop(["types"], inplace=True, axis=1)

# drop other cols
df.drop(["place_id", "plus_code", "reference", "name", "reviews"], inplace=True, axis=1)

In [73]:
embedded_df = pd.read_csv("data_final.csv")

In [74]:
column_names = ['embedding_' + str(i) for i in range(1536)]

# Create DataFrame
embeddings_space = pd.DataFrame(columns=column_names)
# embeddings_space.head()
df = pd.concat([df, embeddings_space], axis=1)
df.fillna(0, inplace=True)

/var/folders/1t/0999bgxs5s17vns3pvsxhgr80000gn/T/ipykernel_25783/248425590.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(0, inplace=True)


In [75]:
result = pd.concat([embedded_df, df], ignore_index=True)
# result.head(-1)

In [76]:
test = result.dropna(axis="columns")

In [80]:
test.to_csv('data-with-null-embeddings.csv', index=False)

In [82]:
test.head(-1)

,rating,user_ratings_total,topics,postal_code,lat,lng,greater_than_50,is_dentist,is_health,is_establishment,...,embedding_1526,embedding_1527,embedding_1528,embedding_1529,embedding_1530,embedding_1531,embedding_1532,embedding_1533,embedding_1534,embedding_1535
0,4.7,253.0,True,7230,36.861774,30.792039,True,True,True,True,...,0.005595,-0.016748,0.071510,-0.023414,-0.036926,-0.015911,-0.015298,0.007039,0.003957,-0.037707
1,4.8,135.0,True,7600,36.893972,30.691002,True,True,True,True,...,0.008192,-0.018630,0.062999,-0.033759,-0.053932,-0.012153,-0.007448,0.004616,0.000669,-0.030039
2,4.8,90.0,True,7050,36.886999,30.683048,True,True,True,True,...,0.007987,-0.007059,0.060596,-0.026944,-0.037144,-0.010489,-0.014042,0.004921,0.009987,-0.047537
3,4.9,216.0,True,7160,36.852699,30.751760,True,True,True,True,...,0.012997,-0.009758,0.062005,-0.037326,-0.040376,-0.008786,-0.010493,0.014760,0.004886,-0.039173
4,4.7,253.0,True,7230,36.861774,30.792039,True,True,True,True,...,0.005595,-0.016748,0.071510,-0.023414,-0.036926,-0.015911,-0.015298,0.007039,0.003957,-0.037707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,0.0,0.0,True,07160,36.852657,30.752765,False,False,True,True,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
826,0.0,0.0,True,07160,36.859125,30.736756,False,False,True,True,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
827,0.0,0.0,True,07160,36.852657,30.752765,False,False,True,True,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
828,0.0,0.0,True,07160,36.859125,30.736756,False,False,True,True,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [84]:
# X = pd.read_csv("data_final.csv")


rating                  0
user_ratings_total      0
topics                  0
postal_code             0
lat                     0
                     ... 
embedding_34478       781
embedding_34479       781
embedding_34480       781
embedding_34481       781
embedding_34482       781
Length: 34494, dtype: int64